In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# from sklearn.model_selection import KFold

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [96]:
data = pd.read_csv("CommonColumn_data.csv")

data.drop(columns=["Referee"], inplace=True)

data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%Y')
data['Days_Since_Start'] = (data['Date'] - data['Date'].min()).dt.days
data.drop(columns=["Date"], inplace=True)

data.head()

,HomeTeam,AwayTeam,HTHG,HTAG,FTHG,FTAG,FTR,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR,Days_Since_Start
0,Charlton,Man City,2,0,4,0,H,17,8,14,4,6,6,13,12,1,2,0,0,0
1,Chelsea,West Ham,1,0,4,2,H,17,12,10,5,7,7,19,14,1,2,0,0,0
2,Coventry,Middlesbrough,1,1,1,3,A,6,16,3,9,8,4,15,21,5,3,1,0,0
3,Derby,Southampton,1,2,2,2,D,6,13,4,6,5,8,11,13,1,1,0,0,0
4,Leeds,Everton,2,0,2,0,H,17,12,8,6,6,4,21,20,1,3,0,0,0


In [18]:
# data = data.drop(columns=['Referee'])
# data.head()

In [97]:
label_encoders = {}
categorical_columns = ["HomeTeam", "AwayTeam", "FTR"]
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col])

In [98]:
scaler = MinMaxScaler()
numeric_columns = data.columns.difference(["FTR"])
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

In [99]:
X = data.drop(columns=["FTR"]).values
y = data["FTR"].values

In [100]:
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [107]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [108]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
results = []

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu', kernel_regularizer=l2(0.001)))
# Reduce the number of LSTM units (e.g., 32 instead of 64) to prevent the model from memorizing the data.
model.add(Dropout(0.3))  

model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01))) # Increase the strength of L2 regularization (l2(0.01) instead of l2(0.001)).
model.add(Dropout(0.5))  #Use a higher dropout rate (e.g., Dropout(0.5)).

model.add(Dense(3, activation='softmax')) 

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [104]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [105]:
from sklearn.utils.class_weight import compute_class_weight


class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping], class_weight=class_weight_dict)


In [106]:
# history = model.fit(X_train, y_train, epochs=200, batch_size=40, validation_data=(X_test, y_test), callbacks=[early_stopping])

history = model.fit(X_train, y_train, epochs=200, batch_size=40, validation_data=(X_test, y_test), callbacks=[early_stopping], class_weight=class_weight_dict)


Epoch 1/200
161/161 [==============================] - 6s 17ms/step - loss: 1.2617 - accuracy: 0.4596 - val_loss: 1.1005 - val_accuracy: 0.7382
Epoch 2/200
161/161 [==============================] - 1s 6ms/step - loss: 0.9719 - accuracy: 0.6398 - val_loss: 0.7390 - val_accuracy: 0.8074
Epoch 3/200
161/161 [==============================] - 1s 6ms/step - loss: 0.7519 - accuracy: 0.7531 - val_loss: 0.5547 - val_accuracy: 0.8897
Epoch 4/200
161/161 [==============================] - 1s 6ms/step - loss: 0.6298 - accuracy: 0.8307 - val_loss: 0.4504 - val_accuracy: 0.9451
Epoch 5/200
161/161 [==============================] - 1s 6ms/step - loss: 0.5330 - accuracy: 0.8870 - val_loss: 0.3700 - val_accuracy: 0.9875
Epoch 6/200
161/161 [==============================] - 1s 6ms/step - loss: 0.4521 - accuracy: 0.9364 - val_loss: 0.3112 - val_accuracy: 0.9988
Epoch 7/200
161/161 [==============================] - 1s 6ms/step - loss: 0.3940 - accuracy: 0.9570 - val_loss: 0.2681 - val_accuracy: 1.000

In [93]:
# history = model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))

In [81]:
# model.save("football_lstm_model.h5")

In [109]:
model.save("football_lstm_model_with_regularization_M2.h5")

C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [54]:
# The probabilities show a very high confidence for class 1 (likely "Draw") and low confidence for others:

# Class probabilities: [[1.7677021e-18 9.9403149e-01 5.9684962e-03]]
# The model is almost certain that the result will be a "Draw" (class 1), with only a tiny chance for other outcomes.



In [110]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

51/51 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       451
           1       1.00      1.00      1.00       385
           2       1.00      1.00      1.00       768

    accuracy                           1.00      1604
   macro avg       1.00      1.00      1.00      1604
weighted avg       1.00      1.00      1.00      1604

[[451   0   0]
 [  0 385   0]
 [  0   0 768]]


In [68]:
# def predict_result(home_team, away_team):
    
#     home_team_encoded = label_encoders["HomeTeam"].transform([home_team])[0]
#     away_team_encoded = label_encoders["AwayTeam"].transform([away_team])[0]

#     # Create a dummy input for prediction
#     input_data = np.zeros((1, X.shape[2]))
#     input_data[0, 0] = home_team_encoded
#     input_data[0, 1] = away_team_encoded
#     # Adjust based on the order of your features

#     # Reshape for LSTM
#     input_data = input_data.reshape((1, 1, X.shape[2]))

#     # Predict
#     prediction = model.predict(input_data)
#     print(f"Class probabilities: {prediction}")
#     result_index = np.argmax(prediction)
#     result_label = label_encoders["FTR"].inverse_transform([result_index])[0]

#     return result_label




# def predict_result(home_team, away_team):
#     home_team_encoded = label_encoders["HomeTeam"].transform([home_team])[0]
#     away_team_encoded = label_encoders["AwayTeam"].transform([away_team])[0]

#     input_data = np.zeros((1, X.shape[2]))
#     input_data[0, 0] = home_team_encoded
#     input_data[0, 1] = away_team_encoded
#     input_data[0, 2] = 0
    
#     input_data = input_data.reshape((1, 1, X.shape[2]))

#     prediction = model.predict(input_data)
    
#     print(f"\nClass probabilities: {prediction}")
#     result_index = np.argmax(prediction)
    
#     if prediction[0][result_index] < 0.7: 
#         return "Uncertain"
    
#     result_label = label_encoders["FTR"].inverse_transform([result_index])[0]
#     return result_label

In [ ]:
def predict_result(home_team, away_team):
    # Encode input teams
    home_team_encoded = label_encoders["HomeTeam"].transform([home_team])[0]
    away_team_encoded = label_encoders["AwayTeam"].transform([away_team])[0]

    # Create a dummy input for prediction, filling all other features with zeros
    input_data = np.zeros((1, X.shape[2]))  # Shape (1, num_features)
    input_data[0, 0] = home_team_encoded
    input_data[0, 1] = away_team_encoded
    input_data[0, 2] = 0  
    

    # Reshape for LSTM
    input_data = input_data.reshape((1, 1, X.shape[2]))  # (samples, timesteps, features)

    prediction = model.predict(input_data)
    print(f"\nClass probabilities: {prediction}")
    
    result_index = np.argmax(prediction)
    result_label = label_encoders["FTR"].inverse_transform([result_index])[0]

    return result_label

In [119]:
home_team = input("Enter the Home Team")
away_team = input("Enter the Away Team")
result = predict_result(home_team, away_team)
print(f"Predicted result: {result}")

Enter the Home Team Man City
Enter the Away Team Liverpool


1/1 [==============================] - 0s 51ms/step

Class probabilities: [[1.4228896e-04 9.9671644e-01 3.1412446e-03]]
Predicted result: D


In [ ]:
# Enter the Home Team Arsenal
# Enter the Away Team Fulham
# 1/1 [==============================] - 0s 47ms/step
# Class probabilities: [[1.7677021e-18 9.9403149e-01 5.9684962e-03]]
# Predicted result: D

SyntaxError: invalid decimal literal (1102738444.py, line 3)

In [25]:
# label_encoders = {}
# for col in ['HomeTeam', 'AwayTeam', 'FTR']:
#     label_encoders[col] = LabelEncoder()
#     data[col] = label_encoders[col].fit_transform(data[col])

In [45]:
original_team = pd.read_csv("CommonColumn_data.csv")

values_list = original_team["AwayTeam"].unique()
print(values_list)

['Man City' 'West Ham' 'Middlesbrough' 'Southampton' 'Everton'
 'Aston Villa' 'Bradford' 'Arsenal' 'Ipswich' 'Newcastle' 'Liverpool'
 'Chelsea' 'Man United' 'Tottenham' 'Charlton' 'Sunderland' 'Derby'
 'Coventry' 'Leicester' 'Leeds' 'Blackburn' 'Bolton' 'Fulham' 'West Brom'
 'Middlesboro' 'Birmingham' 'Wolves' 'Portsmouth' 'Crystal Palace'
 'Norwich' 'Wigan' 'Watford' 'Sheffield United' 'Reading' 'Stoke' 'Hull'
 'Burnley' 'Blackpool' 'Swansea' 'QPR' 'Cardiff' 'Bournemouth'
 'Huddersfield' 'Brighton' 'Brentford']


In [49]:
original_team['FTR'].value_counts()

FTR
H    3667
A    2331
D    2022
Name: count, dtype: int64

In [26]:
# data.fillna(0, inplace=True)

In [27]:
# scaler = MinMaxScaler()
# numerical_cols = ['FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'HY', 'AY', 'HR', 'AR']
# data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

In [28]:
# def create_sequences(data, target_col, seq_length=5):
#     sequences = []
#     targets = []
    
#     for i in range(seq_length, len(data)):
#         seq = data.iloc[i-seq_length:i].drop(columns=[target_col]).values
#         target = data.iloc[i][target_col]
#         sequences.append(seq)
#         targets.append(target)
    
#     return np.array(sequences), np.array(targets)


In [29]:
# target_col = 'FTR'  
# seq_length = 5 
# X, y = create_sequences(data, target_col, seq_length)

In [30]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# model = Sequential([
#     LSTM(64, activation='tanh', input_shape=(seq_length, X.shape[2]), return_sequences=True),
#     Dropout(0.2),
#     LSTM(32, activation='tanh'),
#     Dropout(0.2),
#     Dense(16, activation='relu'),
#     Dense(len(np.unique(y)), activation='softmax') 
# ])

In [32]:
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [33]:
# history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

In [122]:
df = pd.read_csv("CommonColumn_data.csv")
df.head()

,Date,HomeTeam,AwayTeam,HTHG,HTAG,FTHG,FTAG,FTR,Referee,HS,...,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
0,19-08-2000,Charlton,Man City,2,0,4,0,H,Rob Harris,17,...,14,4,6,6,13,12,1,2,0,0
1,19-08-2000,Chelsea,West Ham,1,0,4,2,H,Graham Barber,17,...,10,5,7,7,19,14,1,2,0,0
2,19-08-2000,Coventry,Middlesbrough,1,1,1,3,A,Barry Knight,6,...,3,9,8,4,15,21,5,3,1,0
3,19-08-2000,Derby,Southampton,1,2,2,2,D,Andy D'Urso,6,...,4,6,5,8,11,13,1,1,0,0
4,19-08-2000,Leeds,Everton,2,0,2,0,H,Dermot Gallagher,17,...,8,6,6,4,21,20,1,3,0,0
